In [1]:
from mleap.analyze_results import AnalyseResults
from mleap.data import Data
import pandas as pd
from mleap.data.estimators import instantiate_default_estimators

import matplotlib.pyplot as plt
pd.options.display.max_rows = 1000

/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='delgado_datasets/', 
                        output_h5_predictions_group='experiments/predictions/')


In [3]:
error_all_datasets = analyze.calculate_error_all_datasets(metric='mean_squared_error')

### All datasets

In [4]:
error_all_datasets = analyze.calculate_error_all_datasets(metric='mean_squared_error')

#### Simple average

In [5]:
res_df = analyze.calculate_average_std(error_all_datasets)
res_df

,avg,std
BaggingClassifier,1.046759,2.170619
RandomForestClassifier,1.142671,2.374306
GradientBoostingClassifier,1.468495,3.491370
SVC,2.090772,4.853857
LogisticRegression,2.219204,5.866994
Lasso,2.345978,6.466949
LassoLars,2.418444,6.549009
RidgeRegression,2.423562,6.671876
PassiveAggressiveClassifier,2.781483,6.971238
BernoulliNaiveBayes,3.283329,9.959481


#### Cohen's d

In [6]:
cohens_d = analyze.cohens_d(error_all_datasets)
cohens_d

,Cohen's d
BaggingClassifier-BaselineClassifier,0.487072
BaselineClassifier-RandomForestClassifier,-0.478721
BaselineClassifier-GradientBoostingClassifier,-0.448151
BaggingClassifier-GaussianNaiveBayes,0.419429
GaussianNaiveBayes-RandomForestClassifier,-0.399409
BaselineClassifier-SVC,-0.392251
BaggingClassifier-NeuralNetworkDeepClassifier,0.379588
BaselineClassifier-LogisticRegression,-0.376227
NeuralNetworkDeepClassifier-RandomForestClassifier,-0.367934
BaselineClassifier-Lasso,-0.362688


#### t-test

In [7]:
t_test, t_test_df = analyze.t_test(error_all_datasets)
t_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,-2.577343,0.011255
1,BaggingClassifier - BernoulliNaiveBayes,-1.641961,0.103403
2,BaggingClassifier - GaussianNaiveBayes,-2.219410,0.028474
3,BaggingClassifier - GradientBoostingClassifier,-0.767673,0.444297
4,BaggingClassifier - Lasso,-1.425266,0.156861
5,BaggingClassifier - LassoLars,-1.487784,0.139618
6,BaggingClassifier - LogisticRegression,-1.402536,0.163521
7,BaggingClassifier - NeuralNetworkDeepClassifier,-2.008593,0.046986
8,BaggingClassifier - PassiveAggressiveClassifier,-1.777957,0.078125
9,BaggingClassifier - RandomForestClassifier,-0.223110,0.823856


#### sign test

In [8]:
sign_test, sign_test_df = analyze.sign_test(error_all_datasets)
sign_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,-5.508497,3.619105e-08
1,BaggingClassifier - BernoulliNaiveBayes,-2.309375,2.092278e-02
2,BaggingClassifier - GaussianNaiveBayes,-3.094279,1.972918e-03
3,BaggingClassifier - GradientBoostingClassifier,-0.691396,4.893169e-01
4,BaggingClassifier - Lasso,-1.385625,1.658614e-01
5,BaggingClassifier - LassoLars,-0.835909,4.032062e-01
6,BaggingClassifier - LogisticRegression,-1.303451,1.924209e-01
7,BaggingClassifier - NeuralNetworkDeepClassifier,-2.655073,7.929128e-03
8,BaggingClassifier - PassiveAggressiveClassifier,-1.719988,8.543453e-02
9,BaggingClassifier - RandomForestClassifier,0.093508,9.254996e-01


#### t-test with Bonferroni correction

In [9]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(error_all_datasets)
t_test_bonferroni_df

,pair,p_value
0,BaggingClassifier - BaselineClassifier,0.877912
1,BaggingClassifier - BernoulliNaiveBayes,1.000000
2,BaggingClassifier - GaussianNaiveBayes,1.000000
3,BaggingClassifier - GradientBoostingClassifier,1.000000
4,BaggingClassifier - Lasso,1.000000
5,BaggingClassifier - LassoLars,1.000000
6,BaggingClassifier - LogisticRegression,1.000000
7,BaggingClassifier - NeuralNetworkDeepClassifier,1.000000
8,BaggingClassifier - PassiveAggressiveClassifier,1.000000
9,BaggingClassifier - RandomForestClassifier,1.000000


#### Wilcoxon test

In [10]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(error_all_datasets)
wilcoxon_test_df

,pair,statistic,p_value
0,BaggingClassifier - BaselineClassifier,0.0,5.143754e-11
1,BaggingClassifier - BernoulliNaiveBayes,215.0,1.978773e-06
2,BaggingClassifier - GaussianNaiveBayes,100.0,1.981287e-08
3,BaggingClassifier - GradientBoostingClassifier,239.0,1.196442e-04
4,BaggingClassifier - Lasso,637.0,1.321655e-01
5,BaggingClassifier - LassoLars,761.0,6.027771e-01
6,BaggingClassifier - LogisticRegression,390.0,6.469831e-03
7,BaggingClassifier - NeuralNetworkDeepClassifier,107.0,1.734954e-08
8,BaggingClassifier - PassiveAggressiveClassifier,151.0,9.607049e-07
9,BaggingClassifier - RandomForestClassifier,666.0,8.340890e-01


#### Friedman test

In [11]:
friedman_test, friedman_test_df = analyze.friedman_test(error_all_datasets)
friedman_test_df

,statistic,p_value
0,290.869744,3.869762e-55


In [12]:
nemeniy_test = analyze.nemenyi(error_all_datasets)
pd.DataFrame(nemeniy_test)

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,Lasso,LassoLars,LogisticRegression,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,RidgeRegression,SVC
BaggingClassifier,-1.000000,0.000499,0.957183,0.630756,1.000000,0.999847,1.000000,0.999895,0.833231,0.997361,1.000000,1.000000,0.996546
BaselineClassifier,0.000499,-1.000000,0.340894,0.811800,0.008225,0.041015,0.008724,0.036415,0.600899,0.111008,0.000383,0.007751,0.122993
BernoulliNaiveBayes,0.957183,0.340894,-1.000000,0.999996,0.998778,0.999986,0.998911,0.999978,1.000000,1.000000,0.947278,0.998630,1.000000
GaussianNaiveBayes,0.630756,0.811800,0.999996,-1.000000,0.924988,0.989864,0.928861,0.987618,1.000000,0.998951,0.597226,0.920969,0.999237
GradientBoostingClassifier,1.000000,0.008225,0.998778,0.924988,-1.000000,1.000000,1.000000,1.000000,0.984238,0.999995,0.999999,1.000000,0.999991
Lasso,0.999847,0.041015,0.999986,0.989864,1.000000,-1.000000,1.000000,1.000000,0.999122,1.000000,0.999747,1.000000,1.000000
LassoLars,1.000000,0.008724,0.998911,0.928861,1.000000,1.000000,-1.000000,1.000000,0.985401,0.999996,0.999999,1.000000,0.999992
LogisticRegression,0.999895,0.036415,0.999978,0.987618,1.000000,1.000000,1.000000,-1.000000,0.998827,1.000000,0.999822,1.000000,1.000000
NeuralNetworkDeepClassifier,0.833231,0.600899,1.000000,1.000000,0.984238,0.999122,0.985401,0.998827,-1.000000,0.999969,0.809087,0.983002,0.999981
PassiveAggressiveClassifier,0.997361,0.111008,1.000000,0.998951,0.999995,1.000000,0.999996,1.000000,0.999969,-1.000000,0.996256,0.999993,1.000000


### Per dataset

In [13]:
error_per_dataset, error_per_dataset_df = analyze.calculate_error_per_dataset(metric='mean_squared_error')
error_per_dataset_df

score  \
abalone                      BaggingClassifier              0.488760   
                             BaselineClassifier             1.453227   
                             BernoulliNaiveBayes            0.759971   
                             GaussianNaiveBayes             0.612038   
                             GradientBoostingClassifier     0.498912   
                             Lasso                          0.413882   
                             LassoLars                      0.376236   
                             LogisticRegression             0.511240   
                             NeuralNetworkDeepClassifier    0.556925   
                             PassiveAggressiveClassifier    0.791153   
                             RandomForestClassifier         0.490210   
                             RidgeRegression                0.376844   
                             SVC                            0.432197   
acute_inflammation           BaggingClassifier              0.000000   
                             BaselineClassifier             0.400000   
                             BernoulliNaiveBayes            0.125000   
                             GaussianNaiveBayes             0.175000   
                             GradientBoostingClassifier     0.000000   
                             Lasso                          0.115363   
                             LassoLars                      0.027725   
                             LogisticRegression             0.100000   
                             NeuralNetworkDeepClassifier    0.125000   
                             PassiveAggressiveClassifier    0.100000   
                             RandomForestClassifier         0.000000   
                             RidgeRegression                0.027922   
                             SVC                            0.000000   
acute_nephritis              BaggingClassifier              0.000000   
                             BaselineClassifier             0.425000   
                             BernoulliNaiveBayes            0.025000   
                             GaussianNaiveBayes             0.025000   
                             GradientBoostingClassifier     0.000000   
                             Lasso                          0.054275   
                             LassoLars                      0.026478   
                             LogisticRegression             0.000000   
                             NeuralNetworkDeepClassifier    0.100000   
                             PassiveAggressiveClassifier    0.000000   
                             RandomForestClassifier         0.000000   
                             RidgeRegression                0.027527   
                             SVC                            0.000000   
adult                        BaggingClassifier              0.150701   
                             BaselineClassifier             0.358357   
                             BernoulliNaiveBayes            0.196116   
                             GaussianNaiveBayes             0.191215   
                             GradientBoostingClassifier     0.134570   
                             Lasso                          0.158223   
                             LassoLars                      0.125713   
                             LogisticRegression             0.157774   
                             NeuralNetworkDeepClassifier    0.154796   
                             PassiveAggressiveClassifier    0.159387   
                             RandomForestClassifier         0.137796   
                             RidgeRegression                0.125718   
                             SVC                            0.160814   
annealing                    BaggingClassifier              0.087542   
                             BaselineClassifier             0.925926   
                             BernoulliNaiveBayes            0.387205   
                             Ga